In [90]:
from siphon.catalog import TDSCatalog
from datetime import datetime, timedelta, timezone
import xarray as xr

initial_time = datetime.today().replace(month=9, day=22, minute=0, second= 0, microsecond=0).astimezone(timezone.utc)

window_end = datetime.today().astimezone(timezone.utc)


# Access GFS catalog on THREDDS serverhttps://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_onedeg/catalog.xml
gfs_catalog_url = 'https://thredds.ucar.edu/thredds/catalog/grib/NCEP/HRRR/CONUS_2p5km/catalog.xml'
gfs_cat = TDSCatalog(gfs_catalog_url)

# Choose the 'Full Collection (Reference / Forecast Time) Dataset'
gfs_full_collection = gfs_cat.datasets['Full Collection (Reference / Forecast Time) Dataset']

# Connect to NCSS (NetCDF Subset Service) for the full collection dataset
ncss = gfs_full_collection.subset()

# Build query for data within a time range
query = ncss.query()
#query.lonlat_box(north=50, south=24, east=-66, west=-125)  # CONUS region
query.variables('Temperature_height_above_ground')
# Forecast time window
#query.add_query_parameter(time_start=initial_time.strftime('%Y-%m-%dT%H:%M:%SZ'))
#query.add_query_parameter(time_duration='PT30H')
query.all_times()

# Fetch the data
ncep_gfs_data = ncss.get_data(query)



In [91]:
ncep_gfs_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Originating_or_generating_Center: US National Weather Service, National Centres for Environmental Prediction (NCEP)
    Originating_or_generating_Subcenter: 0
    GRIB_table_version: 2,1
    Type_of_generating_process: Forecast
    Analysis_or_forecast_generating_process_identifier_defined_by_originating_centre: High Resolution Rapid Refresh (HRRR)
    Conventions: CF-1.6
    history: Read using CDM IOSP GribCollection v3
    featureType: GRID
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridCoverageWriter)
Original Dataset = NCEP_HRRR_CONUS_2p5km#TwoD; Translation Date = 2024-09-29T22:59:23.736Z
    geospatial_lat_min: 20.178622167005965
    geospatial_lat_max: 52.81751731155333
    geospatial_lon_min: -130.12289290846252
    geospatial_lon_max: -60.86645211019561
    dimensions(sizes): reftime(1), x(2145), y(1377), time(19), height_above_ground3(1)
    variables

In [103]:
ncep_gfs_data.variables['reftime']

<class 'netCDF4._netCDF4.Variable'>
float64 reftime(reftime)
    units: Hour since 2024-09-27T00:00:00Z
    standard_name: forecast_reference_time
    long_name: GRIB reference time
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [104]:
ncep_gfs_data.variables['reftime'].units

'Hour since 2024-09-27T00:00:00Z'

In [105]:
ncep_gfs_data.variables['time']

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    units: Hour
    standard_name: forecast_period
    long_name: time offset from runtime
    udunits: Hour since 2024-09-27T00:00:00Z
unlimited dimensions: 
current shape = (19,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [107]:
ncep_gfs_data.variables['time'].udunits

'Hour since 2024-09-27T00:00:00Z'

In [93]:
xrd = xr.Dataset(
    data_vars = {
        'Temperature_height_above_ground': (
            ['reftime','time', 'height_above_ground3', 'y', 'x'], 
            ncep_gfs_data.variables['Temperature_height_above_ground'][:] - 273.15)},
    coords={
        'reftime': ncep_gfs_data.variables['reftime'][:],
        'time': ncep_gfs_data.variables['time'][:],
        'height_above_ground3': ncep_gfs_data.variables['height_above_ground3'][:],
        'y': ncep_gfs_data.variables['y'][:],
        'x': ncep_gfs_data.variables['x'][:] })


In [94]:
xrd

<xarray.Dataset>
Dimensions:                          (reftime: 1, time: 19,
                                      height_above_ground3: 1, y: 1377, x: 2145)
Coordinates:
  * reftime                          (reftime) float64 68.0
  * time                             (time) float64 0.0 1.0 2.0 ... 17.0 18.0
  * height_above_ground3             (height_above_ground3) float64 2.0
  * y                                (y) float32 -263.8 -261.3 ... 3.231e+03
  * x                                (x) float32 -2.763e+03 ... 2.682e+03
Data variables:
    Temperature_height_above_ground  (reftime, time, height_above_ground3, y, x) float32 ...

In [101]:
xrd.time

<xarray.DataArray 'time' (time: 19)>
array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18.])
Coordinates:
  * time     (time) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 14.0 15.0 16.0 17.0 18.0